In [2]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import geoviews as gv
# Turn off warning messages
import warnings
warnings.filterwarnings("ignore")

# Import API key
from config import geoapify_key

In [3]:
# Read the WeatherPy_vacation.csv into a DataFrame
vacation_df = pd.read_csv('/Users/taylordacpano/Desktop/Data Bootcamp/World_Weather_Analysis/Weather_database/Vacation_Search/weatherPy_vacation.csv')

# Display sample data
vacation_df

,Hotel Name,City,Country,Max Temp,Lat,Lng,Hotel Name.1
0,0,Esperance,AU,57.60,-33.8667,121.9000,Best Western
1,6,Mahibadhoo,MV,80.76,3.7833,72.9667,Coral View Inn
2,7,Kloulklubed,PW,82.18,7.0419,134.2556,The Adventures Inn
3,8,Punta Arenas,CL,51.82,-53.1500,-70.9167,Hotel Lacolet
4,11,Hobart,AU,80.46,-42.8794,147.3294,The Old Woolstore Apartment Motel
...,...,...,...,...,...,...,...
249,570,Masyaf,SY,43.90,35.0653,36.3406,مصياف السياحي
250,571,Scalea,IT,57.72,39.8130,15.7956,Hotel Scalea
251,574,Tutoia,BR,79.99,-2.7619,-42.2744,Pousada San Vicente
252,576,Palu,ID,75.90,-0.8917,119.8707,Ujung Pandang Hotel


In [4]:
# Configure the map plot
map_plot = vacation_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Max Temp",
    scale = 1,
    color = "City",
    hover_cols = ["Hotel Name", "Country", "Current Description"]
)

# Display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Max Temp,Hotel Name,Country)

In [5]:
# Create DataFrames for each city by filtering the 'vacation_df' using the loc method
vacation_start = vacation_df.loc[(vacation_df["City"] == "Pacific Grove")]
vacation_end = vacation_df.loc[(vacation_df["City"] == "Pacific Grove")]
vacation_stop1 = vacation_df.loc[(vacation_df["City"] == "Hollister")]
vacation_stop2 = vacation_df.loc[(vacation_df["City"] == "Half Moon Bay")]
vacation_stop3 = vacation_df.loc[(vacation_df["City"] == "Laguna")]

In [6]:
# Use the Pandas concat function to create a new DataFrame to store the itinerary details.
itinerary_df = pd.concat([vacation_start, vacation_stop1, vacation_stop2, vacation_stop3, vacation_end])

#display sample data
itinerary_df

,Hotel Name,City,Country,Max Temp,Lat,Lng,Hotel Name.1
30,74,Pacific Grove,US,58.41,36.6177,-121.9166,Pacific Grove Inn
53,125,Hollister,US,58.69,36.8525,-121.4016,Joshua Inn Bed & Breakfast
65,152,Half Moon Bay,US,60.58,37.4636,-122.4286,Half Moon Bay Inn
180,432,Laguna,US,56.14,38.4210,-121.4238,Holiday Inn Express & Suites
30,74,Pacific Grove,US,58.41,36.6177,-121.9166,Pacific Grove Inn


In [7]:
# Create a Pandas DataFrame to store the latitude and longitude for each city in the itineray
waypoints_df = itinerary_df.copy()

# Display sample data
waypoints_df

,Hotel Name,City,Country,Max Temp,Lat,Lng,Hotel Name.1
30,74,Pacific Grove,US,58.41,36.6177,-121.9166,Pacific Grove Inn
53,125,Hollister,US,58.69,36.8525,-121.4016,Joshua Inn Bed & Breakfast
65,152,Half Moon Bay,US,60.58,37.4636,-122.4286,Half Moon Bay Inn
180,432,Laguna,US,56.14,38.4210,-121.4238,Holiday Inn Express & Suites
30,74,Pacific Grove,US,58.41,36.6177,-121.9166,Pacific Grove Inn


In [80]:
# Configure the map plot by using the itineraty_df
waypoints_map = itinerary_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Max Temp",
    scale = 2.5,
    color = "City",
    hover_cols = ["Hotel Name", "City", "Current Description"]
)

# Display the map
waypoints_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Max Temp,Hotel Name)

In [9]:
# Set parameters to trace the route
radius = 5000
params = {
    "mode":"drive",
    "apiKey": geoapify_key,
}# Set an empty waypoints String variable
waypoints = ""

# Iterate through the route_df DataFrame to define the waypoints
for index, row in waypoints_df.iterrows():
    waypoints = waypoints + str(row["Lat"]) + "," + str(row["Lng"]) + "|"

# Delete the last character from the string
waypoints = waypoints[:-1]

# Add the waypoints to the params dictionary
params["waypoints"] = waypoints

# Display the params dictionary
params

{'mode': 'drive',
 'apiKey': '6bf7b77ed2844f80931198e2ff8a94cb',
 'waypoints': '36.6177,-121.9166|36.8525,-121.4016|37.4636,-122.4286|38.421,-121.4238|36.6177,-121.9166'}

In [10]:
# Set up the base URL for the Geoapify Places API.
base_url = "https://api.geoapify.com/v1/routing"

# Make request and retrieve the JSON data by using the params dictionaty
route_response = requests.get(base_url, params=params)

# Convert the API response to JSON format
route_response = route_response.json()
route_response

{'features': [{'type': 'Feature',
   'properties': {'mode': 'drive',
    'waypoints': [{'location': [-121.9166, 36.6177], 'original_index': 0},
     {'location': [-121.4016, 36.8525], 'original_index': 1},
     {'location': [-122.4286, 37.4636], 'original_index': 2},
     {'location': [-121.4238, 38.421], 'original_index': 3},
     {'location': [-121.9166, 36.6177], 'original_index': 4}],
    'units': 'metric',
    'distance': 702633,
    'distance_units': 'meters',
    'time': 25989.971,
    'legs': [{'distance': 68164,
      'time': 2845.937,
      'steps': [{'from_index': 0,
        'to_index': 1,
        'distance': 10,
        'time': 0.822,
        'instruction': {'text': 'Drive west on Pine Avenue.'}},
       {'from_index': 1,
        'to_index': 11,
        'distance': 228,
        'time': 22.71,
        'instruction': {'text': 'Turn right onto 14th Street.'}},
       {'from_index': 11,
        'to_index': 50,
        'distance': 1041,
        'time': 92.003,
        'instructi

In [13]:
# Fetch the route's legs coordinates from the JSON reponse
legs = route_response["features"][0]['geometry']['coordinates']
legs

[[[-121.916402, 36.618153],
  [-121.916516, 36.618185],
  [-121.916476, 36.618278],
  [-121.916466, 36.618303],
  [-121.916124, 36.619096],
  [-121.916121, 36.619103],
  [-121.916097, 36.619159],
  [-121.916073, 36.619213],
  [-121.916069, 36.619224],
  [-121.91577, 36.619913],
  [-121.915704, 36.620064],
  [-121.915679, 36.620121],
  [-121.915632, 36.620101],
  [-121.915583, 36.62008],
  [-121.915337, 36.619977],
  [-121.915255, 36.619921],
  [-121.915228, 36.619908],
  [-121.915209, 36.6199],
  [-121.914765, 36.619711],
  [-121.914271, 36.6195],
  [-121.914133, 36.61944],
  [-121.914045, 36.619402],
  [-121.913949, 36.619367],
  [-121.913859, 36.61934],
  [-121.913777, 36.619318],
  [-121.913661, 36.619296],
  [-121.913445, 36.619266],
  [-121.913223, 36.619248],
  [-121.91268, 36.619196],
  [-121.912627, 36.61919],
  [-121.9122, 36.619144],
  [-121.912138, 36.619137],
  [-121.91161, 36.619081],
  [-121.911077, 36.619026],
  [-121.910537, 36.618972],
  [-121.91028, 36.618947],
  [-12

In [20]:
# Create and empty list to store the longitude of each step
route_lng = []
leglist = []
# Create and empty list to store the latitude of step
route_lat = []
# Loop through the legs coordinates to fetch the latitude and longitude for each step
def removeNestings(legs):
    for i in legs:
        if type(i) == list:
            removeNestings(i)
        else:
            leglist.append(i)
print('The original List: ',legs)
removeNestings(legs)
print('the list after removing nesting: ', leglist)

The original List:  [[[-121.916402, 36.618153], [-121.916516, 36.618185], [-121.916476, 36.618278], [-121.916466, 36.618303], [-121.916124, 36.619096], [-121.916121, 36.619103], [-121.916097, 36.619159], [-121.916073, 36.619213], [-121.916069, 36.619224], [-121.91577, 36.619913], [-121.915704, 36.620064], [-121.915679, 36.620121], [-121.915632, 36.620101], [-121.915583, 36.62008], [-121.915337, 36.619977], [-121.915255, 36.619921], [-121.915228, 36.619908], [-121.915209, 36.6199], [-121.914765, 36.619711], [-121.914271, 36.6195], [-121.914133, 36.61944], [-121.914045, 36.619402], [-121.913949, 36.619367], [-121.913859, 36.61934], [-121.913777, 36.619318], [-121.913661, 36.619296], [-121.913445, 36.619266], [-121.913223, 36.619248], [-121.91268, 36.619196], [-121.912627, 36.61919], [-121.9122, 36.619144], [-121.912138, 36.619137], [-121.91161, 36.619081], [-121.911077, 36.619026], [-121.910537, 36.618972], [-121.91028, 36.618947], [-121.910004, 36.618907], [-121.909484, 36.61881], [-121

In [30]:
# Create an empty DataFrame to store the steps' coordinates
route_df = pd.DataFrame({'Longitude': route_lng,
                         'Latitude': route_lat})
# Add the steps' longitude and latitude from each step as columns to the DataFrame
for item in leglist:
    if item < 0:
        route_lng.append(item)
    else:
        route_lat.append(item)
        

# Display sample data
route_df

,Longitude,Latitude
0,-121.916402,36.618153
1,-121.916516,36.618185
2,-121.916476,36.618278
3,-121.916466,36.618303
4,-121.916124,36.619096
...,...,...
25402,-121.915030,36.617765
25403,-121.915544,36.617910
25404,-121.916025,36.618047
25405,-121.916064,36.618058


In [81]:
# Configure the route path by using the GeoViews' Path function
route_path = gv.Path(route_df).opts(line_width = 2.5, height = 550, color='Red')
route_path

:Path   [Longitude,Latitude]

In [82]:
# Display a composed plot by using the route_map and route_path objects
plot = waypoints_map*route_path
plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Max Temp,Hotel Name)
   .Path.I   :Path   [Longitude,Latitude]